In [2]:
! pip install seqeval

     |████████████████████████████████| 51kB 4.2MB/s eta 0:00:011
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=dd2635648f52cde5489d6e1c5a5a74bc4bd0fdde0e1366c14afea55dd5cbc4a0
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [3]:
pip install transformers

     |████████████████████████████████| 1.8MB 8.3MB/s 
     |████████████████████████████████| 3.2MB 31.9MB/s 
     |████████████████████████████████| 890kB 43.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b9611ed4347eec33c1b2d18cac763f42664beb9eaf294683b0e4e69659891847
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
! pip install pytorch-crf

In [ ]:
! python /content/drive/MyDrive/ColabNotebooks/NLP/nlu/main.py \
        --data_dir=/content/drive/MyDrive/ColabNotebooks/NLP/nlu/data/joint  \
        --model_dir=model \
        --max_seq_len=60 \
        --do_train \
        --do_eval \
        --logging_steps=500

流式输出内容被截断，只能显示最后 5000 行内容。
Iteration:  19% 499/2647 [13:31<49:02,  1.37s/it]
Iteration:  19% 500/2647 [13:32<48:51,  1.37s/it]
Iteration:  19% 501/2647 [13:33<48:39,  1.36s/it]
Iteration:  19% 502/2647 [13:35<48:31,  1.36s/it]
Iteration:  19% 503/2647 [13:36<48:34,  1.36s/it]
Iteration:  19% 504/2647 [13:37<48:36,  1.36s/it]
Iteration:  19% 505/2647 [13:39<48:34,  1.36s/it]
Iteration:  19% 506/2647 [13:40<48:28,  1.36s/it]
Iteration:  19% 507/2647 [13:42<48:23,  1.36s/it]
Iteration:  19% 508/2647 [13:43<48:14,  1.35s/it]
Iteration:  19% 509/2647 [13:44<48:11,  1.35s/it]
Iteration:  19% 510/2647 [13:46<48:10,  1.35s/it]
Iteration:  19% 511/2647 [13:47<48:16,  1.36s/it]
Iteration:  19% 512/2647 [13:48<48:12,  1.35s/it]
Iteration:  19% 513/2647 [13:50<48:05,  1.35s/it]
Iteration:  19% 514/2647 [13:51<47:56,  1.35s/it]
Iteration:  19% 515/2647 [13:52<47:56,  1.35s/it]
Iteration:  19% 516/2647 [13:54<47:55,  1.35s/it]
Iteration:  20% 517/2647 [13:55<47:51,  1.35s/it]
Iteration:  20% 518/264